In [ ]:
import sys
sys.path.insert(0, r'C:\code\astrocam')
from astropy.io import fits
import numpy as np
import cv2
import rawpy
from pathlib import Path
from xisf.xisf_parser import read_xisf
from matplotlib import pyplot as plt
import cv2
import numpy as np


def openImage(fname):
    ext = Path(fname).suffix.lower()

    if ext == '.nef':
      with open(fname, "rb") as f:
        rawimg = rawpy.imread(f)
        img = rawimg.postprocess()
      hdr = None

    elif ext == '.fit':
      f = fits.open(fname)
      ph = f[0]
      img = ph.data
      hdr = ph.header

    elif ext == '.xisf':
      img, hdr = read_xisf(fname)
      img16 = (np.iinfo(np.uint16).max * img).astype(np.uint16)
      img8 = (np.iinfo(np.uint8).max * img).astype(np.uint8)

    # if hdr is not None and hdr['BAYERPAT'] == 'RGGB':
    #   deb = cv2.cvtColor(img, cv2.COLOR_BAYER_BG2RGB)
    #   img = deb.astype(np.float32) / np.iinfo(deb.dtype).max
    #   img = (img * 255).astype(np.uint8)
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    return np.squeeze(img16, axis=2), np.squeeze(img8, axis=2)

In [ ]:
import rawpy
from pathlib import Path

root = Path(r"D:\Astro\Objects\C30\subs")
in_focus = root / "Light_00948_180.0sec_200gain_-0.3C_c_a.xisf"
img16, img8 = openImage(in_focus)



In [ ]:
img16.shape

In [ ]:
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
gray = img8
img = img16

In [ ]:

plt.imshow(gray, cmap='gray')

In [ ]:
Bs = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(7,7))
Bmi = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(21,21))
Be = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(25,25))
Bmo = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(29,29))
d = (Bmo.shape[0] - Bmi.shape[0]) // 2
Bm = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(29,29))
Bm[d:d+Bmi.shape[0], d:d+Bmi.shape[0]] -= Bmi


In [ ]:
K = cv2.morphologyEx(gray, cv2.MORPH_OPEN, Bs)
N = cv2.morphologyEx(cv2.morphologyEx(gray, cv2.MORPH_DILATE, Bm), cv2.MORPH_ERODE, Be)
print(K.shape, N.shape)

In [ ]:
R = K -np.minimum(K,N)

In [ ]:
plt.imshow(R, cmap='gray')

In [ ]:
numstars, labels, stats, centroids = cv2.connectedComponentsWithStats(R, 4, cv2.CV_16U, cv2.CCL_WU)

In [ ]:
centroids

In [ ]:
def getStar(img, staridx):
  centorid_x, centroid_y = centroids[staridx]
  width = stats[staridx, cv2.CC_STAT_WIDTH]
  height = stats[staridx, cv2.CC_STAT_HEIGHT]
  min_row = int(max(0, centroid_y - height))
  max_row = int(min(gray.shape[0], centroid_y + height+1))
  min_col = int(max(0, centorid_x - width))
  max_col = int(min(gray.shape[1], centorid_x + width+1))
  cell = img[min_row:max_row, min_col:max_col]
  return cell

Calculating centroid with higher accuracy
- https://www.lost-infinity.com/night-sky-image-processing-part-4-calculate-the-star-centroid-with-sub-pixel-accuracy/

In [ ]:
star = getStar(img, 1)

Optionally remove background

In [ ]:
bg = np.percentile(img,5)
star = np.clip(star - bg, 0, np.iinfo(img.dtype).max)

In [ ]:
star

1. Calculate the IWC

In [ ]:
cx = np.sum(np.full(star.shape, np.arange(star.shape[1])) * (star**2)) / (np.sum(star**2))
cy = np.sum(np.full(star.shape, np.arange(star.shape[0]).reshape(star.shape[0],1)) * (star**2)) / (np.sum(star**2))
print(cx, cy)
cx = round(cx)
cy = round(cy)
print(star)
print(cx, cy)

Get 3x3 around IWC

In [ ]:
star[cy-1:cy+2, cx-1:cx+2]

2. Round cx, cy to nearest integer and then iteratively improve.

In [ ]:
import math

def calculate_centroid(star: np.ndarray):
    bg = np.percentile(star, 5)
    star = np.clip(star - bg, 0, 255)
    cx = np.sum(np.full(star.shape, np.arange(star.shape[1])) * (star**2)) / (np.sum(star**2))
    cy = np.sum(np.full(star.shape, np.arange(star.shape[0]).reshape(star.shape[0],1)) * (star**2)) / (np.sum(star**2))
    cx = round(cx)
    cy = round(cy)
    inImg = star[cy-1:cy+2, cx-1:cx+2]
    b1 = inImg[0, 0]; a2 = inImg[0, 1]; b2 = inImg[0, 2]
    a1 = inImg[1, 0];  c = inImg[1, 1]; a3 = inImg[1, 2]
    b4 = inImg[2, 0]; a4 = inImg[2, 1]; b3 = inImg[2, 2]

    for i in range(10):
        c2 = 2 * c
        sp1 = (a1 + a2 + c2) / 4
        sp2 = (a2 + a3 + c2) / 4
        sp3 = (a3 + a4 + c2) / 4
        sp4 = (a4 + a1 + c2) / 4
        
        #New maximum is center
        newC = max(sp1, sp2, sp3, sp4)
        
        # Calc position of new center
        ad = math.pow(2.0, -(i + 1.0))

        if (newC == sp1):
            cx = cx - ad # to the left
            cy = cy - ad # to the top

            # Calculate new sub pixel values
            b1n = (a1 + a2 + 2 * b1) / 4
            b2n = (c + b2 + 2 * a2) / 4
            b3n = sp3
            b4n = (b4 + c + 2 * a1) / 4
            a1n = (b1n + c + 2 * a1) / 4
            a2n = (b1n + c + 2 * a2) / 4
            a3n = sp2
            a4n = sp4

        elif (newC == sp2):
            cx = cx + ad # to the right
            cy = cy - ad # to the top

            # Calculate new sub pixel values
            b1n = (2 * a2 + b1 + c) / 4
            b2n = (2 * b2 + a3 + a2) / 4
            b3n = (2 * a3 + b3 + c) / 4
            b4n = sp4
            a1n = sp1
            a2n = (b2n + c + 2 * a2) / 4
            a3n = (b2n + c + 2 * a3) / 4
            a4n = sp3
        elif (newC == sp3):
            cx = cx + ad # to the right
            cy = cy + ad # to the bottom

            # Calculate new sub pixel values
            b1n = sp1
            b2n = (b2 + 2 * a3 + c) / 4
            b3n = (2 * b3 + a3 + a4) / 4
            b4n = (2 * a4 + b4 + c) / 4
            a1n = sp4
            a2n = sp2
            a3n = (b3n + 2 * a3 + c) / 4
            a4n = (b3n + 2 * a4 + c) / 4
        else:
            cx = cx - ad # to the left
            cy = cy + ad # to the bottom   

            # Calculate new sub pixel values
            b1n = (2 * a1 + b1 + c) / 4
            b2n = sp2
            b3n = (c + b3 + 2 * a4) / 4
            b4n = (2 * b4 + a1 + a4) / 4
            a1n = (b4n + 2 * a1 + c) / 4
            a2n = sp1
            a3n = sp3
            a4n = (b4n + 2 * a4 + c) / 4

            c = newC # Oi = Oi+1

            a1 = a1n
            a2 = a2n
            a3 = a3n
            a4 = a4n

            b1 = b1n
            b2 = b2n
            b3 = b3n
            b4 = b4n
    return cx, cy


In [ ]:
cx, cy

In [ ]:
def plotStar3D(ax, star):
  xs = list(range(star.shape[1]))
  ys = list(range(star.shape[0]))
  xs, ys = np.meshgrid(xs, ys)
  X = np.arange(0, star.shape[1], 1)
  Y = np.arange(0, star.shape[0], 1)
  X, Y = np.meshgrid(X, Y)
  ax.plot_surface(xs, ys, star)

In [ ]:
import scipy.optimize as opt

def twoD_GaussianScaledAmp(pos, xo, yo, sigma_x, sigma_y, amplitude, offset):
    """Function to fit, returns 2D gaussian function as 1D array"""
    x,y = pos
    xo = float(xo)
    yo = float(yo)    
    g = offset + amplitude*np.exp( - (((x-xo)**2)/(2*sigma_x**2) + ((y-yo)**2)/(2*sigma_y**2)))
    return g.ravel()

def getFWHM_GaussianFitScaledAmp(img, ax):
    """Get FWHM(x,y) of a blob by 2D gaussian fitting
    Parameter:
        img - image as numpy array
    Returns: 
        FWHMs in pixels, along x and y axes.
    """
    x = np.linspace(0, img.shape[1], img.shape[1])
    y = np.linspace(0, img.shape[0], img.shape[0])
    x, y = np.meshgrid(x, y)
    #Parameters: xpos, ypos, sigmaX, sigmaY, amp, baseline
    initial_guess = (img.shape[1]/2,img.shape[0]/2,10,10,1,0)
    # subtract background and rescale image into [0,1], with floor clipping
    bg = np.percentile(img,5)
    img = np.clip((img - bg) / (img.max() - bg),0,1)

    popt, pcov = opt.curve_fit(twoD_GaussianScaledAmp, (x, y), 
                               img.ravel(), p0=None, #initial_guess,
                               bounds = (
                                   (0, 0, 1, 1, 0.5, -0.1), # Lower bound
                                   (img.shape[1], img.shape[0], img.shape[1], img.shape[0], 1.5, 0.5) # Upper bound
                                )
                            )
    xcenter, ycenter, sigmaX, sigmaY, amp, offset = popt[0], popt[1], popt[2], popt[3], popt[4], popt[5]

    z = offset + amp*np.exp( - (((x-xcenter)**2)/(2*sigmaX**2) + ((y-ycenter)**2)/(2*sigmaY**2)))
    ax.plot_surface(x,y,z)

    FWHM_x = np.abs(4*sigmaX*np.sqrt(-0.5*np.log(0.5)))
    FWHM_y = np.abs(4*sigmaY*np.sqrt(-0.5*np.log(0.5)))
    return (FWHM_x / img.shape[1], FWHM_y / img.shape[0], xcenter, ycenter)



In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
print(f"Num stars: {numstars}")
with PdfPages('fwhm_report_actual.pdf') as pdf:
  for staridx in range(1, numstars):
    star = getStar(img, staridx)

    fig = plt.figure()
    ax = fig.add_subplot(2, 3, 1)
    ax.imshow(star, cmap='gray')

    ax = fig.add_subplot(2, 3, 2, projection='3d')
    plotStar3D(ax, star)

    ax = fig.add_subplot(2, 3, 3, projection='3d')
    cx, cy = calculate_centroid(star)
    FWHM_x, FWHM_y, xcenter, ycenter = getFWHM_GaussianFitScaledAmp(star, ax)

    ax = fig.add_subplot(2, 3, 4)
    ax.axis("off")
    ax.text(0, 0.75, f"FWHM: {FWHM_x:0.3f}, {FWHM_y:0.3f}")

    ax = fig.add_subplot(2, 3, 5)
    ax.axis("off")
    ax.text(0, 0.75, f"Centroid: ({cx:0.3f}, {cy:0.3f})")

    ax = fig.add_subplot(2, 3, 6)
    ax.axis("off")
    ax.text(0, 0.75, f", Curve center: ({xcenter:0.3f}, {ycenter:0.3f})")

    pdf.savefig()
    fig.show()
    plt.close()
    